In [40]:
import json
import re

import requests
from bs4 import BeautifulSoup


def cleanJSON(in_json: str) -> dict:
    """
    Extract airport information from JS variables in PE's coverage map
    
    re match groups:
      0. Airport Code
      1. Airport Name
      2. Airport Class
      3. Coverage Area
    """
    
    description_re = r"<strong>(\w+)</strong>\s+-\s+([\w\s\-\/]+)\(Class\s+(\w)\s+-\s+([\w\s]+)\)"
    
    airport_json = {}
    for airport in in_json:
        match = re.findall(description_re, airport['description'])[0]

        airport_json[match[0]] = {
            'long_name': match[1].strip(),
            'class': match[2],
            'coverage_area': match[3],
            'latitude': airport['lat'],
            'longitude': airport['lng']
        }
        
    return airport_json

def airports_from_map() -> dict:
    """
    Scrape the PE coverage map and return a dictionary of airport information
    """

    PE_MAP_URL = r"https://www.pilotedge.net/map/coverage"

    r = requests.get(PE_MAP_URL)
    map_vars = re.findall(r"Gmaps.map.markers = (.*)", r.text)[0]
    raw_json = json.loads(map_vars[:-1])  # Skip trailing semicolon

    return cleanJSON(raw_json)

In [52]:
def _cached_has_d_atis(airports: dict) -> dict:
    """
    Populate airports dictionary with D-ATIS status from PE's D-ATIS coverage JSON
    """
    
    ATIS_STATUS_URL = r"https://www.pilotedge.net/atis/airports.json"
    
    atis_coverage_json = requests.get(ATIS_STATUS_URL).json()
    atis_coverage = {airport['icao']:airport['has_d_atis'] for airport in atis_coverage_json}
    
    for airport in airports:
        try:
            airports[airport]['has_d_atis'] = atis_coverage[airport]
        except KeyError:
            airports[airport]['has_d_atis'] = False
        
    return airports

In [54]:
airport_coverage = airports_from_map()
airport_coverage = _cached_has_d_atis(airport_coverage)

with open('PE_Airports.json', 'w') as f:
    json.dump(airport_coverage, f, indent=4)